
Example script: explicit simplicial coboundary over a finite ring.

This file is a minimal, fully explicit sanity check for
:func:`homolipop.coboundary.build_coboundary` on a single 2-simplex.

The script constructs the simplicial cochain coboundary

\begin{align}\delta_k : C^k(K;R) \to C^{k+1}(K;R)\end{align}

for a simplicial complex $K$ generated by one triangle and coefficients in
the ring $R = \mathbb Z/p\mathbb Z$.

# Mathematical conventions

## Simplicial complex

Let $K$ be the abstract simplicial complex on vertex set
$\{0,1,2\}$ generated by the 2-simplex

\begin{align}\sigma = (0,1,2).\end{align}

Thus, the simplices of $K$ are exactly

- vertices $(0)$, $(1)$, $(2)$
- edges $(0,1)$, $(0,2)$, $(1,2)$
- the triangle $(0,1,2)$.

The function :func:`homolipop.simplices.build_complex` is assumed to return
a list ``all_simplices`` containing all simplices up to ``max_dim``,
each represented as a strictly increasing tuple of vertex indices.

## Coefficient ring

Fix an integer $p \ge 2$. Define

\begin{align}R = \mathbb Z/p\mathbb Z\end{align}

with addition and additive inverses modulo $p$.
The value ``0`` in the code represents the additive identity in $R$.

## Cochains

For $k \ge 0$, the group of simplicial $k$-cochains with coefficients in $R$ is

\begin{align}C^k(K;R) = \{\varphi : K_k \to R\},\end{align}

where $K_k$ is the set of $k$-simplices of $K$.

## Coboundary

For an ordered $(k+1)$-simplex

\begin{align}\tau = (v_0,\dots,v_{k+1}),\end{align}

its $i$-th face is

\begin{align}d_i\tau = (v_0,\dots,\widehat{v_i},\dots,v_{k+1}).\end{align}

The simplicial coboundary is defined by

\begin{align}(\delta_k\varphi)(\tau)
   =
   \sum_{i=0}^{k+1} (-1)^i\, \varphi(d_i\tau),\end{align}

for all $\varphi \in C^k(K;R)$ and all $(k+1)$-simplices $\tau$.

## Matrix convention used by the implementation

The builder :func:`homolipop.coboundary.build_coboundary` is expected to return a sparse
column representation of the linear maps $\delta_k$ with respect to the basis
of indicator cochains.

For each dimension $k$, the standard basis of $C^k(K;R)$ is indexed by the
$k$-simplices: for a $k$-simplex $\sigma$, let $e_\sigma$ be the cochain

\begin{align}e_\sigma(\sigma)=1,\qquad e_\sigma(\sigma')=0\ \text{for}\ \sigma'\ne\sigma.\end{align}

Then the column of $\delta_k$ corresponding to $e_\sigma$ is the sparse vector
whose entry at a $(k+1)$-simplex $\tau$ equals the coefficient of $e_\tau$
in $\delta_k(e_\sigma)$.

In particular, for $k=0$ one has

\begin{align}\delta_0(e_{(v)})(v_0,v_1)
   =
   e_{(v)}(v_1) - e_{(v)}(v_0),\end{align}

so a vertex basis cochain maps to an alternating sum of incident oriented edges.
Over $\mathbb Z/p\mathbb Z$, the coefficients $-1$ are represented as $p-1$.

## Numerical and ordering contract

This script assumes the following purely structural contracts.

- ``all_simplices`` contains each simplex exactly once.
- A simplex ``s`` has dimension ``len(s) - 1``.
- The coboundary object has attribute ``columns`` where ``columns[k]`` is a list of columns.
  The $j$-th column in ``columns[k]`` corresponds to the $j$-th $k$-simplex
  in the ordering used by :func:`build_coboundary`.
- Each column is a dict mapping global simplex indices to coefficients in $R$.

This file prints the images of basis cochains under $\delta_0$ and $\delta_1$.


In [ ]:
from __future__ import annotations

from typing import Dict, Tuple

from homolipop.coboundary import RingOps, build_coboundary
from homolipop.simplices import build_complex


def main() -> None:
    """
    Construct and print the simplicial coboundary on a single triangle over :math:`\\mathbb Z/p\\mathbb Z`.

    The script performs the following steps.

    1. Fix a modulus ``p`` and define ring operations for :math:`R=\\mathbb Z/p\\mathbb Z`.
    2. Build the simplicial complex generated by the triangle ``(0, 1, 2)`` up to dimension 2.
    3. Build sparse coboundary operators :math:`\\delta_0` and :math:`\\delta_1`.
    4. Print the images of all vertex basis cochains under :math:`\\delta_0`.
    5. Print the images of all edge basis cochains under :math:`\\delta_1`.

    Raises
    ------
    KeyError
        If the internal indexing assumptions described in the module docstring are violated.
        In a correct implementation this should not occur.

    Notes
    -----
    The signs :math:`(-1)^i` in :math:`\\delta_k` are interpreted in the ring :math:`R`.
    Hence ``-1`` is represented as ``p-1`` in the printed output.

    This script does not assert :math:`\\delta_{k+1}\\circ\\delta_k = 0`, but it is a natural
    additional test to add for further validation.
    """
    p = 5

    def add_mod(a: int, b: int) -> int:
        """Return ``a + b`` in :math:`\\mathbb Z/p\\mathbb Z`."""
        return (a + b) % p

    def neg_mod(a: int) -> int:
        """Return ``-a`` in :math:`\\mathbb Z/p\\mathbb Z`."""
        return (-a) % p

    def is_zero_mod(a: int) -> bool:
        """Return ``True`` if and only if ``a`` is the zero element in :math:`\\mathbb Z/p\\mathbb Z`."""
        return (a % p) == 0

    triangles = [(0, 1, 2)]
    complex_data = build_complex(triangles, max_dim=2)

    simplices = complex_data.all_simplices
    filtration_simplices = simplices

    ring = RingOps(one=1, add=add_mod, neg=neg_mod, is_zero=is_zero_mod)
    cob = build_coboundary(simplices, ring=ring)

    by_dim_global: Dict[int, list[int]] = {}
    for global_index, simplex in enumerate(filtration_simplices):
        by_dim_global.setdefault(len(simplex) - 1, []).append(global_index)

    global_to_local_by_dim: Dict[Tuple[int, int], int] = {}
    for dim, global_indices in by_dim_global.items():
        for local_index, global_index in enumerate(global_indices):
            global_to_local_by_dim[(dim, global_index)] = local_index

    print("Filtration simplices with global indices:")
    for i, s in enumerate(filtration_simplices):
        print(i, s)

    print(f"\nCoboundary over Z/{p}Z:")
    print("δ on vertices (0-simplices) gives 1-simplices with signs ±1 mod p.")
    for global_vertex in by_dim_global.get(0, []):
        vertex = filtration_simplices[global_vertex]
        local_vertex = global_to_local_by_dim[(0, global_vertex)]
        column = cob.columns[0][local_vertex]
        image = {filtration_simplices[j]: c for j, c in sorted(column.items())}
        print("δ", vertex, "=", image)

    print("\nδ on edges (1-simplices) gives 2-simplices with signs ±1 mod p.")
    for global_edge in by_dim_global.get(1, []):
        edge = filtration_simplices[global_edge]
        local_edge = global_to_local_by_dim[(1, global_edge)]
        column = cob.columns[1][local_edge]
        image = {filtration_simplices[j]: c for j, c in sorted(column.items())}
        print("δ", edge, "=", image)


if __name__ == "__main__":
    main()